In [1]:
import sympy
sympy.init_printing()

Valve equation
==========

Let's linearise the nasty nonlinear term in the equation percentage valve relationship in T4 Problem 4 (or T2 problem 4)

$$ F = \underbrace{C_v \alpha^{x - 1}}_{\text{nonlinear}} $$

First we introduce the requisite symbols. Notice that we specify constraints on these variables, this will make simplifications better later on.

In [2]:
C_v, alpha, x = sympy.symbols('C_v, alpha, x', positive=True)

In [3]:
term = C_v*alpha**(x - 1)

We also introduce a barred versions of the variable. Sympy automatically constructs these to typesetting nicely.

In [4]:
xbar = sympy.symbols('xbar', positive=True)

For single variable expressions, we can use `sympy.series` to linearise for us. Note that he help for `sympy.series` references the help for `sympy.Expr.series`, which has a lot more detail about the operation of this function

In [5]:
sympy.series?

Signature: sympy.series(expr, x=None, x0=0, n=6, dir='+')
Docstring:
Series expansion of expr around point `x = x0`.

See the doctring of Expr.series() for complete details of this wrapper.
File:      ~/anaconda3/lib/python3.6/site-packages/sympy/series/series.py
Type:      function


In [6]:
sympy.Expr.series?

Signature: sympy.Expr.series(self, x=None, x0=0, n=6, dir='+', logx=None)
Docstring:
Series expansion of "self" around ``x = x0`` yielding either terms of
the series one by one (the lazy series given when n=None), else
all the terms at once when n != None.

Returns the series expansion of "self" around the point ``x = x0``
with respect to ``x`` up to ``O((x - x0)**n, x, x0)`` (default n is 6).

If ``x=None`` and ``self`` is univariate, the univariate symbol will
be supplied, otherwise an error will be raised.

>>> from sympy import cos, exp
>>> from sympy.abc import x, y
>>> cos(x).series()
1 - x**2/2 + x**4/24 + O(x**6)
>>> cos(x).series(n=4)
1 - x**2/2 + O(x**4)
>>> cos(x).series(x, x0=1, n=2)
cos(1) - (x - 1)*sin(1) + O((x - 1)**2, (x, 1))
>>> e = cos(x + exp(y))
>>> e.series(y, n=2)
cos(x + 1) - y*sin(x + 1) + O(y**2)
>>> e.series(x, n=2)
cos(exp(y)) - x*sin(exp(y)) + O(x**2)

If ``n=None`` then a generator of the series terms will be returned.

>>> term=cos(x).series(n=None)
>>> [

Calling series by itself will result in an error term (the one with an $\mathcal{O}$). This is useful to estimate the error of the approximation.

In [7]:
sympy.series(term, x, xbar, 2)

But mostly we will be interested in the expression rather than the error, so we will remove that term with the `removeO` method:

In [8]:
lineq = sympy.series(term, x, xbar, 2).removeO()
lineq

Rewriting in terms of devation variables
--------------

While we are here, we can also rewrite in terms of deviation variables:

In [9]:
xprime = sympy.symbols("x'", positive=True)

In [10]:
lineq_deviation = lineq.subs({x: xprime + xbar})
lineq_deviation

# A note about simplification

You will note that we specified `positive=True` for all our symbols when we created them. This is because the default assumptions about variables in SymPy are that they are complex. And for complex numbers, `log` is not a 1-to-1 function. See if you understand the following:

In [11]:
xbar, alpha = sympy.symbols('xbar, alpha')
sympy.exp(xbar*sympy.log(alpha)).simplify()

In [12]:
xbar, alpha = sympy.symbols('xbar, alpha', positive=True)
sympy.exp(xbar*sympy.log(alpha)).simplify()

Multiple variables
---------------

Unfortunately, SymPy doesn't have a built-in function for multivariate Taylor series, and consecutive application of the `series` function doesn't do exactly what we want.

In [13]:
variables = x, y, z = sympy.symbols('x, y, z')
bars = xbar, ybar, zbar = sympy.symbols('xbar, ybar, zbar')

In [14]:
term = x*y*z

Note that the other variables are assumed to be constant here, so we don't recover the answer we are looking for.

In [15]:
term.series(x, xbar, 2).removeO().series(y, ybar, 2).removeO()

This function applies the rule basically as it is written in the textbook:

In [16]:
def linearise(expr, variables, bars):
    """Linearise a nonlinear expression
    
    Parameters
    ----------
    expr: A sympy expression - this could be a single term or a sum of terms
    variables: The variables in the expression - all the other symbols in the expression will be treated as parameters
    bars: the "barred" versions of the variables
    
    Note: This function will *not work correctly with derivatives in expr*
    """
    vars_and_bars = list(zip(variables, bars))
    # This is the constant term
    result = expr.subs(vars_and_bars)
    # now, we take the derivative with respect to each variable, evaluated at the steady state:
    for variable, bar in vars_and_bars:
        result += (variable - bar)*expr.diff(variable).subs(vars_and_bars)
    return result

In [17]:
linearise(term, variables, bars)